# Tutorial Bismark

### Requerimientos

- Bismark https://github.com/FelixKrueger/Bismark  
- Samtools http://www.htslib.org/  
- trim-galore https://github.com/FelixKrueger/TrimGalore  
- Bowtie2 https://github.com/BenLangmead/bowtie2  


#### Instalación con conda:

 1. Crea y activa un ambiente en conda.

      * `conda create bismark_ejemplo`
      * `conda activate bismark_ejemplo`

 2. Instala los requerimientos:

`conda install -c bioconda bismark trim-galore bowtie2 samtools jupyter-lab

`conda install -c bioconda bismark`  
`conda install -c bioconda trim-galore`  
`conda install -c bioconda samtools`  
`conda install -c conda-forge jupyterlab`* 
`conda install -c bioconda bowtie2`*  

---

#### Software de UCSC

      `bedGraphToBigWig`: Convierte archivos bedGraph a bigWig (binario y acceso rápido a los datos).  
      `bedClip`: Remueve líneas afuera de un cromosoma en un archivo bed.  


 1. Descarga los programas desde la pagina de UCSC: https://hgdownload.soe.ucsc.edu/downloads.html  

      * Linux: http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64.v369/  
      * macOS: http://hgdownload.soe.ucsc.edu/admin/exe/macOSX.x86_64/  


 2. Crea un directorio para estas herramientas:  

      `mkdir -p ~/ucsc_tools`  

 3. Otorga permisos a los programas:  

      `chmod +x bedClip`  
      `chmod +x bedGraphToBigWig`  

 4. Agrega el directorio de los programas al PATH a `bashrc` o `bash_profile`:  

      `export PATH=~/ucsc_tools:$PATH`  

 5. Abre una terminal nueva y confirma ls instalación:  

      `which(bedClip)`  
      `which(bedGraphToBigWig)`  

---

### Datos

 1. Sequencia de tu genoma, un archivo fasta indexado `samtools faidx genome.fa`

      * UCSC https://hgdownload.cse.ucsc.edu/goldenPath/hg38/  
      * igenome illumina https://support.illumina.com/sequencing/sequencing_software/igenome.html  

 2. Tamaño de los chromosomas https://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/
 
 ---
 

---
# Preparar genoma

In [58]:
%%bash
ls -l data/00_util/
printf "\n\n\n"
cat ./bin/01_prepare_genome.sh

total 62620
drwxrwxr-x 4 ahcorcha ahcorcha     4096 jun 30 14:49 Bisulfite_Genome
-rw-r----- 1 ahcorcha ahcorcha 51834845 jun 30 14:34 chr22.fa
-rw-rw-r-- 1 ahcorcha ahcorcha       23 jun 30 14:34 chr22.fa.fai
-rw-r----- 1 ahcorcha ahcorcha 12255678 jun 30 14:34 chr22.fa.gz
-rw-rw-r-- 1 ahcorcha ahcorcha      217 jun 30 15:38 genomic_nucleotide_frequencies.txt
-rw-r----- 1 ahcorcha ahcorcha    11672 jun 30 09:52 hg38.chrom.sizes



#!/bin/bash

genome=../data/00_util/

bismark_genome_preparation ${genome}



In [32]:
%%bash
ls data/00_util/Bisulfite_Genome/
printf "\n\n\n"
ls data/00_util/Bisulfite_Genome/CT_conversion/

CT_conversion
GA_conversion



BS_CT.1.bt2
BS_CT.2.bt2
BS_CT.3.bt2
BS_CT.4.bt2
BS_CT.rev.1.bt2
BS_CT.rev.2.bt2
genome_mfa.CT_conversion.fa


---
# Trimming 


In [2]:
%%bash
cat ./bin/02_trim_fastq.sh

#!/bin/bash

FQ1=../data/01_fastq/K562_1_chr22.fq.gz
FQ2=../data/01_fastq/K562_2_chr22.fq.gz

OUT=../data/02_fastq_trimmed

trim_galore --quality 20 --gzip --paired --phred33 --fastqc \
            --illumina --output_dir ${OUT} \
            --cores 4 ${FQ1} ${FQ2}


In [14]:
%%bash
ls -hgl data/02_fastq_trimmed/ref*


-rw-rw-r-- 1 ahcorcha 1.9K jul  1 20:27 data/02_fastq_trimmed/ref_K562_1_chr22.fq_trimming_report.txt
-rw-rw-r-- 1 ahcorcha 241K jul  1 20:27 data/02_fastq_trimmed/ref_K562_1_chr22_val_1_fastqc.html
-rw-rw-r-- 1 ahcorcha 288K jul  1 20:27 data/02_fastq_trimmed/ref_K562_1_chr22_val_1_fastqc.zip
-rw-rw-r-- 1 ahcorcha 2.0K jul  1 20:27 data/02_fastq_trimmed/ref_K562_2_chr22.fq_trimming_report.txt
-rw-rw-r-- 1 ahcorcha 249K jul  1 20:27 data/02_fastq_trimmed/ref_K562_2_chr22_val_2_fastqc.html
-rw-rw-r-- 1 ahcorcha 300K jul  1 20:27 data/02_fastq_trimmed/ref_K562_2_chr22_val_2_fastqc.zip


---
# Align


In [3]:
%%bash
cat ./bin/03_align.sh

#!/bin/bash


genome=../data/00_util/
mismatches=1
PREFIX=one_mismatch

FQ1=../data/02_fastq_trimmed/K562_1_chr22_val_1.fq.gz
FQ2=../data/02_fastq_trimmed/K562_2_chr22_val_2.fq.gz
OUT=../data/03_aligned


echo "bismark aligned"; date
bismark --bowtie2 -p 2 --parallel 1 --bam --fastq -N ${mismatches} \
 	--output_dir ${OUT} \
        --prefix ${PREFIX} \
        --genome_folder ${genome} \
 	-1 ${FQ1} -2 ${FQ2}


In [17]:
%%bash
ls -l data/03_aligned/
printf "\n\n\n"
cat data/03_aligned/ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_PE_report.txt

total 301568
-rw-rw-r-- 1 ahcorcha ahcorcha 308792209 jun 30 15:12 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.bam
-rw-rw-r-- 1 ahcorcha ahcorcha      1941 jun 30 15:12 one_mismatch.K562_1_chr22_val_1_bismark_bt2_PE_report.txt
-rw-rw-r-- 1 ahcorcha ahcorcha      1941 jul  1 20:24 ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_PE_report.txt

Bismark report for: ./data/02_fastq_trimmed/K562_1_chr22_val_1.fq.gz and ./data/02_fastq_trimmed/K562_2_chr22_val_2.fq.gz (version: v0.22.3)
Bismark was run with Bowtie 2 against the bisulfite genome of /home/ahcorcha/repos/bismark-examples/data/00_util/ with the specified options: -q -N 1 --score-min L,0,-0.2 -p 2 --reorder --ignore-quals --no-mixed --no-discordant --dovetail --maxins 500
Option '--directional' specified (default mode): alignments to complementary strands (CTOT, CTOB) were ignored (i.e. not performed)

Final Alignment report
Sequence pairs analysed in total:	999873
Number of paired-end alignments with a unique best hit:	996885


---
# Deduplicate 

Elimina sequencias duplicadas por PCR.

In [4]:
%%bash
cat ./bin/04_deduplicate.sh

#!/bin/bash


BAM_IN=../data/03_aligned/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.bam
OUT=../data/04_deduplicated

echo "Start deduplication"; date
deduplicate_bismark --paired \
                    --bam  \
                    --output_dir ${OUT} \
                    ${BAM_IN}


In [21]:
%%bash
ls -l data/04_deduplicated/
printf "\n\n\n"
cat data/04_deduplicated/ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplication_report.txt

total 289776
-rw-rw-r-- 1 ahcorcha ahcorcha 296709335 jun 30 15:29 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.bam
-rw-rw-r-- 1 ahcorcha ahcorcha       807 jun 30 15:40 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.nucleotide_stats.txt
-rw-rw-r-- 1 ahcorcha ahcorcha       315 jun 30 15:29 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplication_report.txt
-rw-rw-r-- 1 ahcorcha ahcorcha       807 jul  1 20:29 ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.nucleotide_stats.txt
-rw-rw-r-- 1 ahcorcha ahcorcha       315 jul  1 20:29 ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplication_report.txt


Total number of alignments analysed in ./data/03_aligned/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.bam:	996885
Total number duplicated alignments removed:	40485 (4.06%)
Duplicated alignments were found at:	39055 different position(s)

Total count of deduplicated leftover sequences: 956400 (95.94% of total)



---

# Extract_methylation


In [5]:
%%bash
cat ./bin/05_extract_methylation.sh

#!/bin/bash


BAM_IN=../data/04_deduplicated/one_mismatch*.bam
OUT=../data/05_bismark_extractor/
genome=../data/00_util/

echo "bismark_methylation_extractor"; date
bismark_methylation_extractor --paired-end \
                              --parallel 1 \
                              --ignore_r2 2 \
			      --gzip \
                              --output ${OUT} \
                              --genome_folder ${genome} \
                              ${BAM_IN}


In [62]:
%%bash
ls -l data/05_bismark_extractor/
printf "\n\n\n"
cat data/05_bismark_extractor/ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt
printf "\n\n\n"
head -n 20  data/05_bismark_extractor/ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.M-bias.txt

printf "\n\n\n"
zcat data/05_bismark_extractor/CpG_OB_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz | head

total 171192
-rw-rw-r-- 1 ahcorcha ahcorcha 23342032 jun 30 15:35 CHG_OB_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
-rw-rw-r-- 1 ahcorcha ahcorcha 23362321 jun 30 15:35 CHG_OT_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
-rw-rw-r-- 1 ahcorcha ahcorcha 55334841 jun 30 15:35 CHH_OB_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
-rw-rw-r-- 1 ahcorcha ahcorcha 55620437 jun 30 15:35 CHH_OT_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
-rw-rw-r-- 1 ahcorcha ahcorcha  8796514 jun 30 15:35 CpG_OB_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
-rw-rw-r-- 1 ahcorcha ahcorcha  8768351 jun 30 15:35 CpG_OT_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
-rw-rw-r-- 1 ahcorcha ahcorcha    22567 jun 30 15:35 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.M-bias.txt
-rw-rw-r-- 1 ahcorcha ahcorcha      903 jun 30 15:35 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.d

---
# bismark2bedGraph


In [60]:
%%bash
ls  data/05_bismark_extractor/C*

cat ./bin/06_bismark2bedGraph.sh

data/05_bismark_extractor/CHG_OB_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
data/05_bismark_extractor/CHG_OT_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
data/05_bismark_extractor/CHH_OB_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
data/05_bismark_extractor/CHH_OT_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
data/05_bismark_extractor/CpG_OB_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
data/05_bismark_extractor/CpG_OT_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
#!/bin/bash

echo "bismark2bedGraph"; date
IN=../data/05_bismark_extractor

ls -l ${IN}

OUT=../data/06_bedGraph
name=one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz

bismark2bedGraph --dir ${OUT} \
                 --output ${name} \
                 --buffer_size 10000M \
                 --CX_context \
                 $( ls ${IN}/C*${name} )


Formato BedGraph:  

`<chromosome> <start position> <end position> <methylation percentage>`  

Formato coverage:  
`<chromosome> <start position> <end position> <methylation percentage> <count methylated> <count unmethylated>`  

In [52]:
%%bash
ls -l data/06_bedGraph/
printf "\n\n\n"
echo "<chromosome> <start position> <end position> <methylation percentage>"
zcat data/06_bedGraph/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz | head
printf "\n\n\n"
echo "<chromosome> <start position> <end position> <methylation percentage> <count methylated> <count unmethylated>"
zcat data/06_bedGraph/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz.bismark.cov.gz | head

total 122604
-rw-rw-r-- 1 ahcorcha ahcorcha 66187130 jun 30 15:53 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz
-rw-rw-r-- 1 ahcorcha ahcorcha 59358034 jun 30 15:53 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz.bismark.cov.gz



<chromosome> <start position> <end position> <methylation percentage>
track type=bedGraph
chr22	10510369	10510370	0
chr22	10510386	10510387	0
chr22	10510389	10510390	0
chr22	10510396	10510397	0
chr22	10510398	10510399	0
chr22	10510401	10510402	0
chr22	10510406	10510407	0
chr22	10510413	10510414	0
chr22	10510419	10510420	0



<chromosome> <start position> <end position> <methylation percentage> <count methylated> <count unmethylated>
chr22	10510370	10510370	0	0	1
chr22	10510387	10510387	0	0	1
chr22	10510390	10510390	0	0	1
chr22	10510397	10510397	0	0	1
chr22	10510399	10510399	0	0	1
chr22	10510402	10510402	0	0	1
chr22	10510407	10510407	0	0	1
chr22	10510414	10510414	0	0	1
chr22	10510420	10510420	0	0	1
chr22	10510431	1051043

---
# coverage2cytosine

In [7]:
%%bash
cat ./bin/07_coverage2cytosine.sh

#!/bin/bash

genome=../data/00_util/
COV=../data/06_bedGraph/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz.bismark.cov.gz
OUT=../data/07_cytosine_report
out_cyt_report=one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz.bismark

coverage2cytosine --genome_folder ${genome} \
                  --zero_based \
                  --CX_context \
                  --dir ${OUT} \
                  --output ${out_cyt_report} \
                  ${COV}


In [37]:
%%bash
ls -lhgta data/07_cytosine_report/
printf "\n\n\n"
head data/07_cytosine_report/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz.bismark.CX_report.txt

total 582M
drwxrwxr-x  3 ahcorcha 4.0K jul  1 20:32 .
drwxrwxr-x  2 ahcorcha 4.0K jun 30 16:20 .ipynb_checkpoints
-rw-rw-r--  1 ahcorcha 509M jun 30 15:56 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz.bismark.CX_report.txt
-rw-rw-r--  1 ahcorcha  73M jun 30 15:56 one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz.bismark.CX_report.txt.gz
drwxrwxr-x 10 ahcorcha 4.0K jun 30 14:37 ..



chr22	10510000	-	0	0	CHH	CNN
chr22	10510005	+	0	0	CHH	CTT
chr22	10510008	-	0	0	CHH	CAA
chr22	10510010	-	0	0	CHH	CAC
chr22	10510023	-	0	0	CHH	CTT
chr22	10510026	-	0	0	CHH	CAT
chr22	10510028	+	0	0	CHH	CCT
chr22	10510029	+	0	0	CHH	CTA
chr22	10510038	+	0	0	CHG	CTG
chr22	10510040	-	0	0	CHG	CAG


---
# nucleotide_report

In [8]:
%%bash
cat ./bin/08_nucleotide_report.sh

#!/bin/bash

genome=../data/00_util/
IN=../data/04_deduplicated/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.bam
OUT=../data/04_deduplicated

bam2nuc --genome_folder ${genome} --dir ${OUT} ${IN}


In [44]:
%%bash 
head data/04_deduplicated/ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.nucleotide_stats.txt
tail data/04_deduplicated/ref_one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.nucleotide_stats.txt

(di-)nucleotide	count sample	percent sample	count genomic	percent genomic	coverage
A	56531809	27.86	10382214	26.51	5.445
C	44322407	21.84	9160652	23.39	4.838
G	47151576	23.23	9246186	23.61	5.100
T	54940684	27.07	10370725	26.48	5.298
AA	17133919	8.52	3093542	7.90	5.539
AC	10272118	5.11	2000965	5.11	5.134
AG	15228262	7.57	2937084	7.50	5.185
AT	13523078	6.72	2350607	6.00	5.753
CA	15250914	7.58	2993056	7.64	5.095
CG	2609290	1.30	634646	1.62	4.111
CT	14106824	7.01	2903828	7.42	4.858
GA	12541125	6.24	2444574	6.24	5.130
GC	10291343	5.12	2135821	5.45	4.818
GG	13254714	6.59	2665996	6.81	4.972
GT	10573800	5.26	1999791	5.11	5.287
TA	10858865	5.40	1851025	4.73	5.866
TC	11410531	5.67	2394758	6.12	4.765
TG	15699921	7.81	3008437	7.68	5.219
TT	16333368	8.12	3116495	7.96	5.241


---
# bismark_report

In [9]:
%%bash
cat ./bin/09_bismark_report.sh

#!/bin/bash

genome=../data/00_util/

bismark2report --alignment_report ../data/03_aligned/one_mismatch.K562_1_chr22_val_1_bismark_bt2_PE_report.txt \
	       --dedup_report ../data/04_deduplicated/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplication_report.txt \
	       --splitting_report ../data/05_bismark_extractor/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt \
	       --mbias_report ../data/05_bismark_extractor/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.M-bias.txt \
	       --nucleotide_report ../data/04_deduplicated/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.nucleotide_stats.txt


---
# Format data

In [56]:
%%bash
cat ./bin/10_format_report.sh

#!/bin/bash

cyt_report=../data/07_cytosine_report/one_mismatch.K562_1_chr22_val_1_bismark_bt2_pe.deduplicated.txt.gz.bismark.CX_report.txt
chr_sizes=../data/00_util/hg38.chrom.sizes




sort --output=${cyt_report}.sorted --parallel=2 --buffer-size=5G -k 1,1 -k2,2n ${cyt_report}




awk '{FS="\t";OFS="\t"} { print $1,$2,$2+1,$4 }' ${cyt_report}.sorted  > ${cyt_report}.sorted.meth  




# echo "bedClip meth."; date
# bedClip -verbose=2 \
#         ${cyt_report}.sorted.meth \
#         ${chr_sizes} \
#         ${cyt_report}.sorted.meth.clipped

echo "bedGraphToBigWig meth."; date
bedGraphToBigWig ${cyt_report}.sorted.meth \
		 ${chr_sizes} \
		 ${cyt_report}.sorted.meth.bw; date




echo "Unmeth"; date
awk '{FS="\t";OFS="\t"} { print $1,$2,$2+1,$5 }' \
    ${cyt_report}.sorted  > ${cyt_report}.sorted.unmeth

ls -lahgt ${cyt_report}.sorted.unmeth

# echo "bedClip unmeth."; date
# bedClip -verbose=2 \
#         ${cyt_report}.sorted.unmeth \
#         ${chr_sizes} \
#         ${cyt_report}.